## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [2]:
path_counts_ran = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/nyc_per_user_emoji_counts.csv'
path_users_ran = '/home/stelios/Desktop/Honours Project/Samples/user_data/nyc.csv'
counts_df_ran = pd.read_csv(path_counts_ran,index_col=None, header=0, low_memory = False)
users_df_ran = pd.read_csv(path_users_ran,index_col=None, header=0, low_memory = False)

In [3]:
counts_df = counts_df_ran 

# Fill NaN with 0.0
counts_df = counts_df.fillna(0.0)

In [4]:
# Feature % representation
for index, row in counts_df.iterrows():
    counts_df.loc[index] = counts_df.loc[index]/counts_df.loc[index].sum()
counts_df.describe()

,🦝,🈂,🕍,⛹🏾,🕟,🏌🏿‍♂️,🤹🏼‍♀️,🧛🏾‍♀️,🚵🏼‍♂️,👩🏻‍🚒,...,🇬🇺,🇶🇦,🇻🇬,🇹🇨,🇦🇮,🇭🇰,🇩🇿,🇰🇼,🇧🇹,🇳🇵
count,6.807000e+03,6807.000000,6807.000000,6.807000e+03,6.807000e+03,6.807000e+03,6.807000e+03,6807.000000,6.807000e+03,6.807000e+03,...,6.807000e+03,6807.000000,6807.000000,6807.000000,6807.000000,6807.000000,6807.000000,6.807000e+03,6.807000e+03,6807.000000
mean,1.581352e-07,0.000003,0.000004,2.069121e-07,4.548223e-07,4.548223e-07,4.548223e-07,0.000001,6.769935e-07,9.545653e-08,...,7.835072e-07,0.000002,0.000009,0.000009,0.000009,0.000016,0.000004,9.892767e-07,2.992008e-07,0.000006
std,1.304687e-05,0.000263,0.000319,1.707119e-05,3.752491e-05,3.752491e-05,3.752491e-05,0.000090,5.585504e-05,7.875597e-06,...,6.464290e-05,0.000150,0.000735,0.000735,0.000735,0.001080,0.000303,8.161983e-05,2.468543e-05,0.000527
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
50%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
75%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
max,1.076426e-03,0.021739,0.026316,1.408451e-03,3.095975e-03,3.095975e-03,3.095975e-03,0.007463,4.608295e-03,6.497726e-04,...,5.333333e-03,0.012346,0.060606,0.060606,0.060606,0.086957,0.025000,6.734007e-03,2.036660e-03,0.043478


In [5]:
counts_df['gender'] = users_df_ran['gender']
counts_df['ethnicity'] = users_df_ran['ethnicity']

In [6]:
counts_df = counts_df.dropna()

In [7]:
counts_df.groupby('gender').size()

gender
female    2043
male      1639
dtype: int64

In [8]:
counts_df.groupby('ethnicity').size()

ethnicity
asian        122
black       1116
hispanic      75
white       2369
dtype: int64

In [9]:
## count again all 
counts_all_df = counts_df_ran 

# Fill NaN with 0.0
counts_all_df = counts_all_df.fillna(0.0)

# Feature % representation
for index, row in counts_all_df.iterrows():
    counts_all_df.loc[index] = counts_all_df.loc[index]/counts_all_df.loc[index].sum()
counts_all_df.describe()

,🦝,🈂,🕍,⛹🏾,🕟,🏌🏿‍♂️,🤹🏼‍♀️,🧛🏾‍♀️,🚵🏼‍♂️,👩🏻‍🚒,...,🇬🇺,🇶🇦,🇻🇬,🇹🇨,🇦🇮,🇭🇰,🇩🇿,🇰🇼,🇧🇹,🇳🇵
count,6.807000e+03,6807.000000,6807.000000,6.807000e+03,6.807000e+03,6.807000e+03,6.807000e+03,6807.000000,6.807000e+03,6.807000e+03,...,6.807000e+03,6807.000000,6807.000000,6807.000000,6807.000000,6807.000000,6807.000000,6.807000e+03,6.807000e+03,6807.000000
mean,1.581352e-07,0.000003,0.000004,2.069121e-07,4.548223e-07,4.548223e-07,4.548223e-07,0.000001,6.769935e-07,9.545653e-08,...,7.835072e-07,0.000002,0.000009,0.000009,0.000009,0.000016,0.000004,9.892767e-07,2.992008e-07,0.000006
std,1.304687e-05,0.000263,0.000319,1.707119e-05,3.752491e-05,3.752491e-05,3.752491e-05,0.000090,5.585504e-05,7.875597e-06,...,6.464290e-05,0.000150,0.000735,0.000735,0.000735,0.001080,0.000303,8.161983e-05,2.468543e-05,0.000527
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
50%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
75%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
max,1.076426e-03,0.021739,0.026316,1.408451e-03,3.095975e-03,3.095975e-03,3.095975e-03,0.007463,4.608295e-03,6.497726e-04,...,5.333333e-03,0.012346,0.060606,0.060606,0.060606,0.086957,0.025000,6.734007e-03,2.036660e-03,0.043478


In [10]:
counts_df = counts_df.dropna()

In [11]:
counts_df.shape

(3682, 2223)